Import packages

In [1]:
from pathlib import Path
import numpy as np
import os
from glob import glob
import natsort

Read the images

In [32]:
# SAVE_PATH = Path("Processed-Heart-Detection")
# dcm_path = ROOT_PATH/str(patient_id) # joining paths
# dcm_path = dcm_path.with_suffix(".dcm") # adds suffix

ROOT_PATH = Path(Path(os.getcwd())/"../challenge1")
TRAIN_PATH = ROOT_PATH/"train"
VAL_PATH = ROOT_PATH/"val"

print(TRAIN_PATH)

c:\master\udg\CAD\project\notebooks\..\challenge1\train


In [38]:
train_nevus = sorted(glob(str(TRAIN_PATH/'nevus/*')))
train_others = sorted(glob(str(TRAIN_PATH/'others/*')))
# print(train_nevus)

['c:\\master\\udg\\CAD\\project\\notebooks\\..\\challenge1\\train\\nevus\\nev00001.jpg', 'c:\\master\\udg\\CAD\\project\\notebooks\\..\\challenge1\\train\\nevus\\nev00002.jpg', 'c:\\master\\udg\\CAD\\project\\notebooks\\..\\challenge1\\train\\nevus\\nev00003.jpg', 'c:\\master\\udg\\CAD\\project\\notebooks\\..\\challenge1\\train\\nevus\\nev00004.jpg', 'c:\\master\\udg\\CAD\\project\\notebooks\\..\\challenge1\\train\\nevus\\nev00005.jpg', 'c:\\master\\udg\\CAD\\project\\notebooks\\..\\challenge1\\train\\nevus\\nev00006.jpg', 'c:\\master\\udg\\CAD\\project\\notebooks\\..\\challenge1\\train\\nevus\\nev00007.jpg', 'c:\\master\\udg\\CAD\\project\\notebooks\\..\\challenge1\\train\\nevus\\nev00008.jpg', 'c:\\master\\udg\\CAD\\project\\notebooks\\..\\challenge1\\train\\nevus\\nev00009.jpg', 'c:\\master\\udg\\CAD\\project\\notebooks\\..\\challenge1\\train\\nevus\\nev00010.jpg', 'c:\\master\\udg\\CAD\\project\\notebooks\\..\\challenge1\\train\\nevus\\nev00011.jpg', 'c:\\master\\udg\\CAD\\project\

In [39]:
# Pre-processing

In [ ]:
# Image augmentation
# 1. Create dataset class
# 2. pass the augmentation as an argument (create class/file)